In [46]:
import os
import random
import time
from itertools import permutations

DATA_DIR = "./Data"  # Adjust if your input files are in a different directory

def read_input(file_name):
    """Reads and parses the input file."""
    file_path = os.path.join(DATA_DIR, file_name)
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    # First line contains the number of paintings
    n = int(lines[0].strip())
    paintings = []
    
    # Process each painting line
    for i, line in enumerate(lines[1:], start=0):
        parts = line.strip().split()
        orientation = parts[0]
        num_tags = int(parts[1])
        tags = set(parts[2:])  # Use a set to ensure uniqueness of tags
        paintings.append({"ID": i, "Orientation": orientation, "Tags": tags})
    
    return n, paintings


In [47]:
def create_frameglasses(paintings):
    """
    Generates frameglasses from paintings:
    - Each Landscape (L) is in its own frameglass.
    - Each frameglass for Portraits (P) combines exactly two paintings.
    - Tags in a frameglass are unique.
    """
    landscapes = [p for p in paintings if p["Orientation"] == "L"]
    portraits = [p for p in paintings if p["Orientation"] == "P"]

    frameglasses = []

    # Add each Landscape as its own frameglass
    for landscape in landscapes:
        frameglasses.append({
            "Paintings": [landscape["ID"]],  # Only one Landscape painting per frameglass
            "Tags": landscape["Tags"]  # Tags remain as-is
        })

    # Group every two Portraits together
    for i in range(0, len(portraits), 2):
        if i + 1 < len(portraits):  # Ensure there is a pair
            combined_tags = portraits[i]["Tags"] | portraits[i + 1]["Tags"]  # Combine unique tags
            frameglasses.append({
                "Paintings": [portraits[i]["ID"], portraits[i + 1]["ID"]],
                "Tags": combined_tags
            })
        else:
            # If there is an odd number of Portraits, leave the last one ungrouped (optional)
            frameglasses.append({
                "Paintings": [portraits[i]["ID"]],
                "Tags": portraits[i]["Tags"]
            })

    return frameglasses


In [48]:
def calculate_score(frameglasses):
    """Calculates the Global Robotic Satisfaction."""
    score = 0
    for i in range(len(frameglasses) - 1):
        tags1 = frameglasses[i]["Tags"]
        tags2 = frameglasses[i + 1]["Tags"]

        # Calculate Local Robotic Satisfaction
        common_tags = len(tags1 & tags2)
        tags_in_f1_not_in_f2 = len(tags1 - tags2)
        tags_in_f2_not_in_f1 = len(tags2 - tags1)
        local_score = min(common_tags, tags_in_f1_not_in_f2, tags_in_f2_not_in_f1)
        score += local_score

    return score


In [49]:
def generate_orders(frameglasses):
    """Generates different orders for the frameglasses."""
    # Same order
    same_order = frameglasses[:]

    # Reverse order
    reverse_order = list(reversed(frameglasses))

    # Random order
    random_order = frameglasses[:]
    random.shuffle(random_order)

    # Order by number of tags
    sorted_order = sorted(frameglasses, key=lambda x: len(x["Tags"]), reverse=True)

    return {
        "Same Order": same_order,
        "Reverse Order": reverse_order,
        "Random Order": random_order,
        "Sorted by Tags": sorted_order,
    }


In [50]:
def write_output(frameglasses, output_file_name):
    """Writes the frameglasses to an output file."""
    with open(output_file_name, "w") as file:
        file.write(f"{len(frameglasses)}\n")
        for fg in frameglasses:
            file.write(" ".join(map(str, fg["Paintings"])) + "\n")


In [51]:
def measure_execution_time(func, *args):
    """Measures the execution time of a function."""
    start_time = time.time()
    result = func(*args)
    elapsed_time = time.time() - start_time
    print(f"{func.__name__} executed in {elapsed_time:.4f} seconds")
    return result, elapsed_time


In [52]:
def main():
    input_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".txt")]

    total_score = 0

    for input_file in input_files:
        print(f"Processing {input_file}...")

        # Step 1: Parse input
        n, paintings = read_input(input_file)

        # Step 2: Create frameglasses
        frameglasses = create_frameglasses(paintings)

        # Step 3: Generate different orders
        orders = generate_orders(frameglasses)

        # Step 4: Evaluate and write each order
        for order_name, fg_order in orders.items():
            output_file_name = f"{input_file}_{order_name.replace(' ', '_')}.txt"
            write_output(fg_order, output_file_name)

            # Calculate score
            score = calculate_score(fg_order)
            print(f"Score for {order_name}: {score}")

            # Accumulate total score
            total_score += score

    print(f"Total Global Score: {total_score}")

if __name__ == "__main__":
    main()


Processing 0_example.txt...
Score for Same Order: 1
Score for Reverse Order: 1
Score for Random Order: 1
Score for Sorted by Tags: 1
Processing 10_computable_moments.txt...
Score for Same Order: 159
Score for Reverse Order: 159
Score for Random Order: 153
Score for Sorted by Tags: 181
Processing 110_oily_portraits.txt...
Score for Same Order: 118534
Score for Reverse Order: 118534
Score for Random Order: 118368
Score for Sorted by Tags: 123896
Processing 11_randomizing_paintings.txt...
Score for Same Order: 191059
Score for Reverse Order: 191059
Score for Random Order: 174641
Score for Sorted by Tags: 212753
Processing 1_binary_landscapes.txt...
Score for Same Order: 12
Score for Reverse Order: 12
Score for Random Order: 15
Score for Sorted by Tags: 9
Total Global Score: 1249548
